Citation

In [1]:
import ee,json,os,colour,math,csv,geemap
import numpy as np
import geopandas as geopandas
import pandas as pd
import datetime
ee.Authenticate()
ee.Initialize()
import time
#https://code.earthengine.google.com/0701d40fb1f844cfd376e8e6cbd7d946

Enter verification code: 4/1AfgeXvuOXg04ugBwBkxWY3IA8jaI0AgjJ7Rf8iZYljonqjU0HTIK0WAopbg

Successfully saved authorization token.


### Variables

In [2]:
in_shp = r'E:\GIS\Thesis\Intermediate Data\Z_AOIs\AOI_Paper.shp'
AOI = geemap.shp_to_ee(in_shp)
#def get_s2_sr_cld_col(aoi, start_date, end_date):

C:\code\envs\geemap\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [3]:
def indexJoin(collectionA, collectionB, propertyName):
    #joins cloud prob image to Sen2 SR
    joined = ee.ImageCollection(ee.Join.saveFirst(propertyName).apply(
    primary = collectionA,
    secondary = collectionB,
    condition = ee.Filter.equals(
      leftField = 'system:index',
      rightField = 'system:index')))
    return joined.map(lambda image: image.addBands(ee.Image(image.get(propertyName))))
def maskClouds(img, MAX_CLOUD_PROBABILITY=10):
    #removes cloudy pixels using sen2 cloudprob
    clouds = ee.Image(img.get('cloud_probability')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)
def s2_clear_sky(image):
    #selects only clear sky pixels using sen2 SCL
    scl = image.select('SCL')
    clear_sky_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7)).Or(scl.eq(11)).Or(scl.eq(0)).Or(scl.eq(2))
    return image.updateMask(clear_sky_pixels)
def printCollectionSize(collection):
    print('Size of image collection:')
    print(len(collection['features']))
    print('\n')
def ictime (collection):
    return image.set('dateDist',ee.Number(image.get('system:time_start')).subtract(date_of_interest.millis()).abs())

s2c = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
# S2Cloudless for the cloud probability band.
s2c = s2c.filterBounds(AOI).filterDate('2019-08-01', '2022-08-31') \
.filter(ee.Filter.calendarRange(8, 8,'month')).sort('dateDist')
# S2 L2A for surface reflectance bands.
s2Sr = s2Sr.filterBounds(AOI).filterDate('2019-08-01', '2022-08-31') \
.filter(ee.Filter.calendarRange(8, 8,'month')) \
.filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 10).sort('dateDist')

s2Sr_SCL=ee.ImageCollection(s2Sr.map(s2_clear_sky)) #applys clear sky mask
withCloudProbability = indexJoin(s2Sr_SCL, s2c, 'cloud_probability') #joins cloud prob
masked_aug = ee.ImageCollection(withCloudProbability.map(maskClouds))#applies cloud mask
nimages = masked_aug.size().getInfo()
print(nimages)

18


In [4]:
s2c = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
# S2Cloudless for the cloud probability band.
s2c = s2c.filterBounds(AOI).filterDate('2019-07-01', '2022-08-31') \
.filter(ee.Filter.calendarRange(7, 7,'month')).sort('dateDist')
# S2 L2A for surface reflectance bands.
s2Sr = s2Sr.filterBounds(AOI).filterDate('2019-07-01', '2022-08-31') \
.filter(ee.Filter.calendarRange(7, 7,'month')) \
.filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 10).sort('dateDist')

s2Sr_SCL=ee.ImageCollection(s2Sr.map(s2_clear_sky)) #applys clear sky mask
withCloudProbability = indexJoin(s2Sr_SCL, s2c, 'cloud_probability') #joins cloud prob
masked_july = ee.ImageCollection(withCloudProbability.map(maskClouds))#applies cloud mask
nimages = masked_july.size().getInfo()
print(nimages)

9


In [5]:
type(masked_aug)

ee.imagecollection.ImageCollection

In [6]:
#cleaning for median reducer
#combined_img_meadian=masked.median().clip(AOI)
#combined_img_meadian=combined_img_meadian.select(['B2','B3','B4','B5','B6','B7','B8','B11','B12'])

#cleaning for image stack
s2_aug_stack=masked_aug.mosaic().clip(AOI).select(['B2','B3','B4','B5','B6','B7','B8','B11','B12'])
s2_july_stack=masked_july.mosaic().clip(AOI).select(['B2','B3','B4','B5','B6','B7','B8','B11','B12'])

combo_stack=ee.ImageCollection([s2_july_stack,s2_aug_stack]).mosaic()

#can used median for median reducer
Sen2vis = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4,}
Sen2vis_p = {'bands': ['B4_p90', 'B3_p90', 'B2_p90'],'min': 0,'max': 3000,'gamma': 1.4,}
Map = geemap.Map()
#Map.addLayer(combined_img_meadian, Sen2vis, 'median')
Map.addLayer(combo_stack, Sen2vis, 'combo')
Map.addLayer(s2_aug_stack, Sen2vis, 'Aug')
Map.addLayer(s2_july_stack, Sen2vis, 'July')
Map.addLayerControl()
Map.centerObject(AOI,10)
Map

Map(center=[52.747633082881386, -67.10133488692263], controls=(WidgetControl(options=['position', 'transparent…

In [7]:
#If large image use this
aoi_g= AOI.geometry()
geemap.ee_export_image_to_drive(
    s2_aug_stack, description='Sen2_AOI_Paper_harmonized_Stacked_08_2022', folder='export', region=aoi_g, scale=10, 
)


Exporting Sen2_AOI_Paper_harmonized_Stacked_08_2022 ...
